# FPL with genetic algorithms

Todo - pycall to get data

In [1]:
using CSV
using StatsBase
using Statistics
using DataFrames

In [2]:
df = DataFrame(CSV.File("/home/chickpea/Documents/fpl/data.csv"))
first(df, 3)

,first_name,second_name,team,team_code,id,element_type,now_cost
,String,String,Int64,Int64,Int64,Int64,Float64
1,Mesut,Özil,1,3,1,3,6.8
2,Sokratis,Papastathopoulos,1,3,2,2,4.9
3,David,Luiz Moreira Marinho,1,3,3,2,5.5


In [3]:
# function fitness(ppg, form)
#     return ppg * form
# end

In [4]:
df["fitness"] = df.ep_next
# df["fitness"] = fitness.(df.points_per_game, df.form)
first(df, 3)

,first_name,second_name,team,team_code,id,element_type,now_cost
,String,String,Int64,Int64,Int64,Int64,Float64
1,Mesut,Özil,1,3,1,3,6.8
2,Sokratis,Papastathopoulos,1,3,2,2,4.9
3,David,Luiz Moreira Marinho,1,3,3,2,5.5


In [5]:
# df = df[df.fitness .>= 0.5, :]

In [6]:
id_to_player = Dict()
for (k, v) in zip(df.id, df.second_name)
    id_to_player[k] = v
end
id_to_cost = Dict()
for (k, v) in zip(df.id, df.now_cost)
    id_to_cost[k] = v
end
id_to_element_type = Dict()
for (k, v) in zip(df.id, df.element_type)
    id_to_element_type[k] = v
end
id_to_team = Dict()
for (k, v) in zip(df.id, df.team)
    id_to_team[k] = v
end
id_to_fitness = Dict()
for (k, v) in zip(df.id, df.fitness)
    id_to_fitness[k] = v
end

In [7]:
goalkeepers = df[df.element_type .== 1, :]
defenders = df[df.element_type .== 2, :]
midfielders = df[df.element_type .== 3, :]
attackers = df[df.element_type .== 4, :]
first(attackers, 10)

,first_name,second_name,team,team_code,id,element_type,now_cost,points_per_game
,String,String,Int64,Int64,Int64,Int64,Float64,Float64
1,Alexandre,Lacazette,1,3,6,4,8.5,5.5
2,Edward,Nketiah,1,3,22,4,5.7,2.4
3,Wesley,Moraes,2,7,48,4,6.0,0.0
4,Mbwana,Samatta,2,7,49,4,5.9,0.0
5,Keinan,Davis,2,7,50,4,4.4,1.0
6,Indiana,Vassilev,2,7,51,4,4.4,0.0
7,Ollie,Watkins,2,7,514,4,6.0,6.2
8,Glenn,Murray,3,36,53,4,4.9,0.0
9,Florin,Andone,3,36,64,4,4.9,0.0


In [8]:
function generate_formation()
    n_keepers = 1
    n_defenders = rand(3:5)
    n_midfielders = rand(2:5)
    n_attackers = rand(1:3)
    while sum((n_keepers, n_defenders, n_midfielders, n_attackers)) != 11
        n_defenders = rand(3:5)
        n_midfielders = rand(2:5)
        n_attackers = rand(1:3)
    end
    return n_keepers, n_defenders, n_midfielders, n_attackers
end

generate_formation (generic function with 1 method)

In [9]:
function generate_squad_initial()
    gk_ids = sample(goalkeepers.id, 2, replace=false)
    defender_ids = sample(defenders.id, 5, replace=false)
    midfielder_ids = sample(midfielders.id, 5, replace=false)
    attacker_ids = sample(attackers.id, 3, replace=false)
    vec = vcat(gk_ids, defender_ids, midfielder_ids, attacker_ids)
    return vec
end

generate_squad_initial (generic function with 1 method)

In [10]:
function generate_team_from_squad(squad_vector)
    """Sorts a team into the first 11 parts"""
    gk, d, m, a = generate_formation()
    team_gks = sample(squad_vector[1:2], 1)
    team_defs = sample(squad_vector[3:7], d, replace=false)
    team_mids = sample(squad_vector[8:12], m, replace=false)
    team_atts = sample(squad_vector[13:15], a, replace=false)

    team_vec = vcat(team_gks, team_defs, team_mids, team_atts)
    sub_vec = zeros(4)
    sub_counter = 1
    for id in squad_vector
        position = id_to_element_type[id]
        if position == 1
            if !(id in team_gks)
                sub_vec[sub_counter] = id
                sub_counter += 1
            end
        elseif position == 2
            if !(id in team_defs)
                sub_vec[sub_counter] = id
                sub_counter += 1
            end
        elseif position == 3
            if !(id in team_mids)
                sub_vec[sub_counter] = id
                sub_counter += 1
            end
        else
            if !(id in team_atts)
                sub_vec[sub_counter] = id
                sub_counter += 1
            end
        end
    end
    vec = vcat(team_vec, sub_vec)
    vec = floor.(Int64, vec)
    return vec
end

generate_team_from_squad (generic function with 1 method)

In [11]:
function print_squad_names(squad_vector)
    for i in squad_vector
        println(i, " ", id_to_player[i])
    end
end

print_squad_names (generic function with 1 method)

In [12]:
function squad_cost(squad_vector)
    sum = 0
    if 0 in squad_vector
        println(squad_vector)
    end
    for id in squad_vector
        sum += id_to_cost[id]
    end
    return sum
end

squad_cost (generic function with 1 method)

In [13]:
squad = generate_squad_initial()
squad = generate_team_from_squad(squad)
squad_cost(squad)

75.6

In [14]:
function team_fitness(squad_vector)
    fit = 0
    for id in squad_vector[1:11]
        fit += id_to_fitness[id]
    end
    fit_subs = 0
    for id in squad_vector[12:15]
        fit_subs += id_to_fitness[id] / 2
    end
    return fit + 0.5 * fit_subs
end

team_fitness (generic function with 1 method)

In [15]:
function most_from_one_squad(squad_vector)
    teams = zero(squad_vector)
    for (i, id) in enumerate(squad_vector)
        teams[i] = id_to_team[id]
    end
    uq_teams = unique(teams)
    max = 0
    for team in uq_teams
        bool = teams .== team
        count = sum(bool)
        if count > max
            max = count
        end
    end
    return max
end

most_from_one_squad (generic function with 1 method)

In [16]:
pool = zeros(Int64, (100, 15))
fitnesses = zeros(100);

In [17]:
function validate_player_types(squad_vector)
    types = zeros(size(squad_vector, 1))
    for (i, id) in enumerate(squad_vector)
        types[i] = id_to_element_type[id]
    end
    counts = zeros(4)
    for t in collect(1:4)
        counts[t] = sum(types .== t)
    end
    if sum(counts) > 15
        return false
    end
    if !(0 < counts[1] < 3)
        return false
    elseif !(0 < counts[2] < 6)
        return false
    elseif !(0 < counts[3] < 6)
        return false
    elseif !(0 < counts[4] < 4)
        return false
    end

    team = squad_vector[1:11]
    team_types = zeros(11)
    counts = zeros(4)
    for (i, id) in enumerate(team)
        team_types[i] = id_to_element_type[id]
    end
    for t in collect(1:4)
        counts[t] = sum(team_types .== t)
    end
    if sum(counts) != 11
        return false
    end
    if counts[1] != 1
        return false
    elseif !(3 <= counts[2] <= 5)
        return false
    elseif !(2 <= counts[3] <= 5)
        return false
    elseif !(1 <= counts[4] <= 3)
        return false
    else
        return true
    end
end

validate_player_types (generic function with 1 method)

In [18]:
test = []
append!(test, 1)
append!(test, 2)

2-element Array{Any,1}:
 1
 2

In [19]:
function players_unique(squad_vector)
    found = []
    for id in squad_vector
        if id in found
            return false
        else
            append!(found, id)
        end
    end
    return true
end

players_unique (generic function with 1 method)

In [20]:
function get_player_types(squad_vector)
    types = zeros(size(squad_vector, 1))
    for (i, id) in enumerate(squad_vector)
        types[i] = id_to_element_type[id]
    end
    counts = zeros(4)
    for t in collect(1:4)
        counts[t] = sum(types .== t)
    end
    println("Squad types: ", counts)
    team = squad_vector[1:11]
    team_types = zeros(11)
    counts = zeros(4)
    for (i, id) in enumerate(team)
        team_types[i] = id_to_element_type[id]
    end
    for t in collect(1:4)
        counts[t] = sum(team_types .== t)
    end
    return counts
end

get_player_types (generic function with 1 method)

In [21]:
function print_player_types(squad_vector::Array{Int64, 1})
    counts = get_player_types(squad_vector)
    println(counts)
end

print_player_types (generic function with 1 method)

In [22]:
function validate_squad(squad_vector::Array{Int64,1})
    if squad_cost(squad_vector) <= 100
        cost_flag = true
    else
        cost_flag = false
    end
    if most_from_one_squad(squad_vector) <= 3
        team_flag = true
    else
        team_flag = false
    end
    types_flag = validate_player_types(squad_vector)
    players_unique_flag = players_unique(squad_vector)        
    
    valid = cost_flag * team_flag * types_flag * players_unique_flag
    return valid
end

validate_squad (generic function with 1 method)

In [23]:
function validate_squad(squad_vector::Array{Int64,1}, chosen_team::Array{Int64, 1}, n_changes::Int)
    valid_flag = validate_squad(squad_vector)
    changes_flag = check_changes(chosen_team, squad_vector, n_changes)
    valid = valid_flag * changes_flag
    return valid
end

validate_squad (generic function with 2 methods)

In [24]:
function iterate_valid_team()
    valid = 0
    while valid == 0
        squad = generate_squad_initial()
        squad = generate_team_from_squad(squad)
        valid = validate_squad(squad)
        if valid == 1
            return squad
        end
    end
end

iterate_valid_team (generic function with 1 method)

In [25]:
created = 0
while created < 100
    squad = iterate_valid_team()
    pool[created+1, :] = squad
    fit = team_fitness(squad)
    fitnesses[created+1] = fit
    created += 1
end


In [26]:
function select_best_rows(pool, fitnesses, n_rows)
    weights = fitnesses / sum(fitnesses)
    selected = sample(collect(1:size(pool, 1)), Weights(weights), n_rows, replace=true)
    selected_rows = pool[selected, :]
    selected_fitnesses = fitnesses[selected]
    return selected_rows, selected_fitnesses
end

select_best_rows (generic function with 1 method)

In [27]:
function breed(parent1, parent2, parent3)
    c1, c2 = sort(sample(collect(2:14), 2))
    new = vcat(parent1[1:c1], parent2[c1+1:c2], parent3[c2+1:15])
    return new
end

breed (generic function with 1 method)

In [28]:
println(collect(1:1))

[1]


In [29]:
function subs_swap(squad_vector::Array{Int64, 1})
    gk_chance = rand()
    if gk_chance < 0.1
        tmp = squad_vector[1]
        squad_vector[1] = squad_vector[12]
        squad_vector[12] = tmp
    end
    for i in collect(1:3)
        sub = squad_vector[12+i]
        sub_chance = rand()
        if sub_chance < 0.1
            pos_to_swap = sample(collect(2:11))
            squad_vector[12+i] = squad_vector[pos_to_swap]
            squad_vector[pos_to_swap] = sub
        end
    end
    return squad_vector
end

subs_swap (generic function with 1 method)

In [30]:
function mutate(squad)
    n_to_mutate = sample(1:3)
    pos_to_mutate = sample(collect(1:15), n_to_mutate, replace=false)
    for ind in pos_to_mutate
        id = squad[ind]
        if id_to_element_type[id] == 1
            squad[ind] = sample(goalkeepers.id)
        else
            squad[ind] = sample(df.id)
        end
    end
    return squad
end

mutate (generic function with 1 method)

In [31]:
function iterate_breed(parents)
    valid = 0
    its = 0
    while valid == 0 & its < 1000
        parent_inds = sample(collect(1:50), 3, replace=false)
        p1 = parents[parent_inds[1], :]
        p2 = parents[parent_inds[2], :]
        p3 = parents[parent_inds[3], :]
        new_squad = breed(p1, p2, p3)
        mutate_chance = rand()
        if mutate_chance < 0.3
            new_squad = mutate(new_squad)
        end
        sub_chance = rand()
        if sub_chance < 0.3
            new_squad = subs_swap(new_squad)
        end
        valid = validate_squad(new_squad)
        if valid
            return new_squad
        end
        its += 1
    end
    error("Max iterations exceeded")
end

iterate_breed (generic function with 1 method)

In [32]:
function iterate_breed(parents, chosen_team::Array{Int64,1}, n_changes::Int)
    valid = 0
    its = 0
    while valid == 0 & its < 1000
        parent_inds = sample(collect(1:50), 3, replace=false)
        p1 = parents[parent_inds[1], :]
        p2 = parents[parent_inds[2], :]
        p3 = parents[parent_inds[3], :]
        new_squad = breed(p1, p2, p3)
        mutate_chance = rand()
        if mutate_chance < 0.3
            new_squad = mutate(new_squad)
        end
        sub_chance = rand()
        if sub_chance < 0.3
            new_squad = subs_swap(new_squad)
        end
        valid = validate_squad(new_squad, chosen_team, n_changes)
        if valid
            return new_squad
        end
        its += 1
    end
    error("Max iterations exceeded")
end

iterate_breed (generic function with 2 methods)

In [33]:
pool_size = 200
generations = 1000
early_stop_lim = 200

pool = zeros(Int64, (pool_size, 15))
fitnesses = zeros(pool_size);
max_pool = zero(pool)
max_fitnesses = zero(fitnesses)

created = 0
while created < pool_size
    squad = iterate_valid_team()
    pool[created+1, :] = squad
    fit = team_fitness(squad)
    fitnesses[created+1] = fit
    created += 1
end

max_fitness = 0
early_stop_count = 0

for generation in collect(1:generations)
    println("Generation: ", generation)
    new_pool = zero(pool)
    new_fitnesses = zero(fitnesses)
    parents, parent_fitness = select_best_rows(pool, fitnesses, 50)
    created=0
    while created < pool_size
        new_squad = iterate_breed(parents)
        new_pool[created+1, :] = new_squad
        new_fitnesses[created+1] = team_fitness(new_squad)
        created += 1
    end
    pool = new_pool
    fitnesses = new_fitnesses
    println("Max fitness: ", maximum(fitnesses))
    if maximum(fitnesses) > max_fitness
        max_fitness = maximum(fitnesses)
        max_pool = pool
        max_fitnesses = fitnesses
        early_stop_count = 0
    elseif early_stop_count <= early_stop_lim
        early_stop_count += 1
    else
        println("Early stopping")
        break
    end
end

Generation: 1
Max fitness: 38.800000000000004
Generation: 2
Max fitness: 36.6
Generation: 3
Max fitness: 43.4
Generation: 4
Max fitness: 48.525
Generation: 5
Max fitness: 48.075
Generation: 6
Max fitness: 49.425
Generation: 7
Max fitness: 44.725
Generation: 8
Max fitness: 48.62499999999999
Generation: 9
Max fitness: 45.574999999999996
Generation: 10
Max fitness: 41.300000000000004
Generation: 11
Max fitness: 42.375
Generation: 12
Max fitness: 43.550000000000004
Generation: 13
Max fitness: 45.2
Generation: 14
Max fitness: 45.2
Generation: 15
Max fitness: 43.97500000000001
Generation: 16
Max fitness: 48.275
Generation: 17
Max fitness: 46.925000000000004
Generation: 18
Max fitness: 46.87500000000001
Generation: 19
Max fitness: 46.67500000000001
Generation: 20
Max fitness: 48.60000000000001
Generation: 21
Max fitness: 48.95
Generation: 22
Max fitness: 47.05000000000001
Generation: 23
Max fitness: 47.97500000000001
Generation: 24
Max fitness: 53.07500000000001
Generation: 25
Max fitness: 48

Max fitness: 61.45
Generation: 203
Max fitness: 62.3
Generation: 204
Max fitness: 62.3
Generation: 205
Max fitness: 62.3
Generation: 206
Max fitness: 64.175
Generation: 207
Max fitness: 62.3
Generation: 208
Max fitness: 60.2
Generation: 209
Max fitness: 61.4
Generation: 210
Max fitness: 60.2
Generation: 211
Max fitness: 59.8
Generation: 212
Max fitness: 60.0
Generation: 213
Max fitness: 63.9
Generation: 214
Max fitness: 60.0
Generation: 215
Max fitness: 61.05
Generation: 216
Max fitness: 58.5
Generation: 217
Max fitness: 60.0
Generation: 218
Max fitness: 59.275
Generation: 219
Max fitness: 60.0
Generation: 220
Max fitness: 60.0
Generation: 221
Max fitness: 60.6
Generation: 222
Max fitness: 61.725
Generation: 223
Max fitness: 61.4
Generation: 224
Max fitness: 60.0
Generation: 225
Max fitness: 60.175000000000004
Generation: 226
Max fitness: 60.6
Generation: 227
Max fitness: 61.125
Generation: 228
Max fitness: 60.574999999999996
Generation: 229
Max fitness: 61.85
Generation: 230
Max fitne

Max fitness: 68.675
Generation: 414
Max fitness: 69.07499999999999
Generation: 415
Max fitness: 66.07499999999999
Generation: 416
Max fitness: 66.14999999999999
Generation: 417
Max fitness: 66.77499999999999
Generation: 418
Max fitness: 66.62499999999999
Generation: 419
Max fitness: 66.12499999999999
Generation: 420
Max fitness: 67.29999999999998
Generation: 421
Max fitness: 67.02499999999999
Generation: 422
Max fitness: 68.79999999999998
Generation: 423
Max fitness: 67.29999999999998
Generation: 424
Max fitness: 69.1
Generation: 425
Max fitness: 66.99999999999999
Generation: 426
Max fitness: 66.975
Generation: 427
Max fitness: 67.29999999999998
Generation: 428
Max fitness: 67.29999999999998
Generation: 429
Max fitness: 70.24999999999999
Generation: 430
Max fitness: 67.24999999999999
Generation: 431
Max fitness: 67.04999999999998
Generation: 432
Max fitness: 67.04999999999998
Generation: 433
Max fitness: 68.075
Generation: 434
Max fitness: 67.1
Generation: 435
Max fitness: 66.674999999

In [34]:
best_team = max_pool[argmax(max_fitnesses), :]
print_squad_names(best_team)

325 Darlow
389 Aurier
41 Mings
508 Rodríguez
376 Bednarek
458 Boly
390 Son
276 Sterling
284 Foden
65 March
514 Watkins
8 Leno
425 O'Shea
366 Ings
539 Delap


In [35]:
team_fitness(best_team)

72.89999999999999

In [36]:
squad_cost(best_team)

94.1

In [37]:
typeof(best_team)

Array{Int64,1}

In [38]:
print_player_types(best_team)

Squad types: [2.0, 5.0, 5.0, 3.0]
[1.0, 4.0, 5.0, 1.0]


#### Optimisation of squad

Change one/two players

In [39]:
function check_changes(chosen_team, new_vector, n_changes=1::Int)
    counter = 0
    for id in new_vector
        if id in chosen_team
            counter += 1
        end
    end
    # If we find 11 players, rem is 4, 4 changes
    changes = 15 - counter
    if changes > n_changes
        return false
    else
        return true
    end
end

check_changes (generic function with 2 methods)

In [40]:
best_team = max_pool[argmax(max_fitnesses), :]
print_squad_names(best_team)

325 Darlow
389 Aurier
41 Mings
508 Rodríguez
376 Bednarek
458 Boly
390 Son
276 Sterling
284 Foden
65 March
514 Watkins
8 Leno
425 O'Shea
366 Ings
539 Delap


In [41]:
# chosen_team = [12,
#     41,
#     498,
#     46,
#     110,
#     508,
#     445,
#     105,
#     141,
#     164,
#     224,
#     325,
#     435,
#     137,
#     506]
# print_squad_names(chosen_team)

In [42]:
# Fergus' team
chosen_team = [
    131,
    200,
    255,
    259,
    302,
    254,
    508,
    284,
    262,
    117,
    184,
    99,
    457,
    449,
    146
]
print_squad_names(chosen_team)

131 McCarthy
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
184 Mitrović
99 Peacock-Farrell
457 Doherty
449 Soucek
146 Mitchell


In [62]:
pool_size = 200
generations = 1000
early_stop_lim = 200
n_changes = 1

base_pool = zeros(Int64, (pool_size, 15))
fitnesses = zeros(pool_size);
max_pool = zero(pool)
max_fitnesses = zero(fitnesses)

for i in 1:pool_size
    base_pool[i, :] = chosen_team
end

fitnesses[:] .= team_fitness(base_pool[1, :])

max_fitness = team_fitness(base_pool[1, :])
early_stop_count = 0

for generation in collect(1:generations)
    println("Generation: ", generation)
    new_pool = zero(base_pool)
    new_fitnesses = zero(fitnesses)
    parents, parent_fitness = select_best_rows(base_pool, fitnesses, 50)
    created = 0
    while created < pool_size
        new_squad = iterate_breed(parents, chosen_team, n_changes)
        new_pool[created+1, :] = new_squad
        new_fitnesses[created+1] = team_fitness(new_squad)
        created += 1
    end
    base_pool = new_pool
    fitnesses = new_fitnesses
    println("Maximum fitness: ", maximum(fitnesses))
    if maximum(fitnesses) > max_fitness
        max_fitness = maximum(fitnesses)
        max_pool = base_pool
        max_fitnesses = fitnesses
        early_stop_count = 0
    elseif early_stop_count <= early_stop_lim
        early_stop_count += 1
    else
        println("Early stopping")
        break
    end
end

Generation: 1
Maximum fitness: 61.275
Generation: 2
Maximum fitness: 58.275
Generation: 3
Maximum fitness: 58.275
Generation: 4
Maximum fitness: 58.275
Generation: 5
Maximum fitness: 60.525
Generation: 6
Maximum fitness: 58.275
Generation: 7
Maximum fitness: 58.275
Generation: 8
Maximum fitness: 58.275
Generation: 9
Maximum fitness: 58.275
Generation: 10
Maximum fitness: 58.275
Generation: 11
Maximum fitness: 58.275
Generation: 12
Maximum fitness: 58.275
Generation: 13
Maximum fitness: 58.275
Generation: 14
Maximum fitness: 46.574999999999996
Generation: 15
Maximum fitness: 48.0
Generation: 16
Maximum fitness: 48.0
Generation: 17
Maximum fitness: 48.974999999999994
Generation: 18
Maximum fitness: 50.4
Generation: 19
Maximum fitness: 48.974999999999994
Generation: 20
Maximum fitness: 49.275
Generation: 21
Maximum fitness: 48.974999999999994
Generation: 22
Maximum fitness: 48.974999999999994
Generation: 23
Maximum fitness: 48.974999999999994
Generation: 24
Maximum fitness: 48.97499999999

Maximum fitness: 48.824999999999996
Generation: 193
Maximum fitness: 46.875
Generation: 194
Maximum fitness: 48.0
Generation: 195
Maximum fitness: 48.974999999999994
Generation: 196
Maximum fitness: 48.824999999999996
Generation: 197
Maximum fitness: 48.824999999999996
Generation: 198
Maximum fitness: 46.574999999999996
Generation: 199
Maximum fitness: 49.125
Generation: 200
Maximum fitness: 48.824999999999996
Generation: 201
Maximum fitness: 48.824999999999996
Generation: 202
Maximum fitness: 48.0
Generation: 203
Maximum fitness: 48.875
Early stopping


In [54]:
print_squad_names(chosen_team)
team_fitness(chosen_team)

131 McCarthy
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
184 Mitrović
99 Peacock-Farrell
457 Doherty
449 Soucek
146 Mitchell


53.474999999999994

In [63]:
best_team = max_pool[argmax(max_fitnesses), :]
print_squad_names(best_team)

12 Martínez
200 Dallas
255 Robertson
259 Alexander-Arnold
302 Borges Fernandes
254 Salah
508 Rodríguez
284 Foden
262 Brewster
117 Werner
184 Mitrović
99 Peacock-Farrell
457 Doherty
449 Soucek
146 Mitchell


In [59]:
team_fitness(max_pool[argmax(max_fitnesses), :])

58.175

In [52]:
id_to_fitness[12]

7.8

In [53]:
id_to_fitness[131]

0.0